In [2]:
import pandas as pd
import numpy as np
import math
import itertools

# Initial values
x_E1_base = 140
x_E2_base = 100
x_S1_base = 23
x_S2_base = 17
x_B1_base = 31
max_capacity = 274 * 1.4  # Maximum capacity with overbooking

# Define ranges (±20% of base values)
def get_range(base_value):
  min_val = max(1, math.floor(base_value * 0.8))  # Ensure positive integers
  max_val = math.ceil(base_value * 1.2)
  return range(min_val, max_val + 1)

# Define ticket prices and other parameters
# These are placeholders - you might want to adjust these based on your actual data
r_E1 = 2728000  # Economy Standard
r_E2 = 3582000  # Economy Flexible
r_S1 = 3957000  # Special Economy Standard
r_S2 = 4497000  # Special Economy Flexible
r_B1 = 5860000  # Business Standard

# Compensation factor (percentage of ticket price)
boi_thuong = 200  # 30% of ticket price

# Calculate compensation costs
c_E1 = boi_thuong * r_E1 * 0.01
c_E2 = boi_thuong * r_E2 * 0.01
c_S1 = boi_thuong * r_S1 * 0.01
c_S2 = boi_thuong * r_S2 * 0.01
c_B1 = boi_thuong * r_B1 * 0.01

# Number of seats in each class
S_E = 211
S_S = 35
S_B = 28
S_total = S_E + S_S + S_B

# Seat allocation percentages
S_E1_percent = 60
S_S1_percent = 60
S_B1_percent = 100

# Calculate seat allocations
S_E1 = math.floor(S_E1_percent * S_E * 0.01)
S_E2 = S_E - S_E1
S_S1 = math.floor(S_S1_percent * S_S * 0.01)
S_S2 = S_S - S_S1
S_B1 = S_B * S_B1_percent * 0.01

S_levels = [S_E1, S_E2, S_S1, S_S2, S_B1]

# Show-up probabilities
p_E1 = 0.9
p_E2 = 0.85
p_S1 = 0.9
p_S2 = 0.85
p_B1 = 0.9

p_levels = [p_E1, p_E2, p_S1, p_S2, p_B1]
r_levels = [r_E1, r_E2, r_S1, r_S2, r_B1]
c_levels = [c_E1, c_E2, c_S1, c_S2, c_B1]

# Function to calculate profit
def calculate_profit(x_E1, x_E2, x_S1, x_S2, x_B1):
    # Revenue
    revenue = (r_E1 * x_E1 + r_E2 * x_E2 +
              r_S1 * x_S1 + r_S2 * x_S2 +
              r_B1 * x_B1)
    
    # Compensation cost calculation
    over_E1 = max(p_E1 * x_E1 - S_E1, 0)
    over_E2 = max(p_E2 * x_E2 - S_E2, 0)
    over_S1 = max(p_S1 * x_S1 - S_S1, 0)
    over_S2 = max(p_S2 * x_S2 - S_S2, 0)
    over_B1 = max(p_B1 * x_B1 - S_B1, 0)
    
    comp_cost_E1 = over_E1 * c_E1
    comp_cost_E2 = over_E2 * c_E2
    comp_cost_S1 = over_S1 * c_S1
    comp_cost_S2 = over_S2 * c_S2
    comp_cost_B1 = over_B1 * c_B1
    
    total_comp_cost = (comp_cost_E1 + comp_cost_E2 +
                      comp_cost_S1 + comp_cost_S2 +
                      comp_cost_B1)
    
    # Profit
    profit = revenue - total_comp_cost
    
    return profit

# Create ranges for each ticket type
x_E1_range = get_range(x_E1_base)
x_E2_range = get_range(x_E2_base)
x_S1_range = get_range(x_S1_base)
x_S2_range = get_range(x_S2_base)
x_B1_range = get_range(x_B1_base)

# Initialize results list
results = []
count = 0
# Generate all combinations and calculate profit
for x_E1, x_E2, x_S1, x_S2, x_B1 in itertools.product(
    x_E1_range, x_E2_range, x_S1_range, x_S2_range, x_B1_range
):
    # Check if the combination meets the capacity constraint
    total_tickets = x_E1 + x_E2 + x_S1 + x_S2 + x_B1
    count += 1
    
    if total_tickets <= max_capacity:
        # Calculate profit for this combination
        profit = calculate_profit(x_E1, x_E2, x_S1, x_S2, x_B1)
        
        # Add to results
        results.append({
            'x_E1': x_E1,
            'x_E2': x_E2,
            'x_S1': x_S1,
            'x_S2': x_S2,
            'x_B1': x_B1,
            'total_tickets_sold': total_tickets,
            'profit': profit
        })

# Convert to DataFrame
df_results = pd.DataFrame(results)

# Sort by profit (descending)
df_results = df_results.sort_values('profit', ascending=False)
# Save only the first 5000 records to CSV
df_results.head(5000).to_csv('ticket_allocation_results.csv', index=False)

# Print top 10 results
print("Top 10 most profitable allocations:")
print(df_results.head(10))
print(f"\nTotal combinations checked: {count}")

# Print the best solution
if not df_results.empty:
    best_solution = df_results.iloc[0]
    print("\nBest Solution:")
    print(f"Economy Standard (E1): {best_solution['x_E1']}")
    print(f"Economy Flexible (E2): {best_solution['x_E2']}")
    print(f"Special Economy Standard (S1): {best_solution['x_S1']}")
    print(f"Special Economy Flexible (S2): {best_solution['x_S2']}")
    print(f"Business Standard (B1): {best_solution['x_B1']}")
    print(f"Total Tickets: {best_solution['total_tickets_sold']}")
    print(f"Profit: {best_solution['profit']:,.2f} VND")

Top 10 most profitable allocations:
         x_E1  x_E2  x_S1  x_S2  x_B1  total_tickets_sold        profit
1735222   140   100    23    17    31                 311  1.085193e+09
1735207   140   100    23    16    31                 310  1.084743e+09
1735357   140   100    24    17    31                 312  1.084401e+09
1735342   140   100    24    16    31                 311  1.083952e+09
1796107   141   100    23    17    31                 312  1.083010e+09
1736707   140   101    23    17    31                 312  1.082685e+09
1796092   141   100    23    16    31                 311  1.082561e+09
1674337   139   100    23    17    31                 310  1.082465e+09
1736692   140   101    23    16    31                 311  1.082236e+09
1796242   141   100    24    17    31                 313  1.082219e+09

Total combinations checked: 3470445

Best Solution:
Economy Standard (E1): 140.0
Economy Flexible (E2): 100.0
Special Economy Standard (S1): 23.0
Special Economy Flexible 